# 1. Introduction
This notebook aims to develop and train a U-Net–based deep learning model for organ detection and segmentation in computed tomography (CT) images.

# 2. The U-Net Architecture
U-Net is one of the most common deep learning architectures for segmentation tasks. The block diagram of this model is depicted in the figure below:

<div style="text-align: center;">
    <img src="images/u-net-architecture.png" width="600"/>
</div>

The architecture features a "U" shape, consisting of two main stages: the contracting (encoder) and expansive (decoder) stages.

* The **encoder** captures the context and high-level features of the input image by using several convolutional layers. It gradually reduces the spatial dimensions while increasing the feature dimensions.

* The **decoder** is responsible for reconstructing the output image, which, in the case of segmentation, is the mask that identifies the objects of interest. This stage involves upsampling across the same number of levels as the encoder, followed by convolutional operations to "expand" the contracted image.

One of the unique characteristics of U-Net is its **skip connections**, which link the encoder and decoder stages at each level by merging features. While the contracting and expanding paths ("U" shape) capture high-level contextual information, the skip connections help preserve low-level spatial details that might be lost during downsampling.

# 3. Importing Libraries

In [ ]:
# Generic libraries
import os
import torch
import random
import glob
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
from pathlib import Path
from torchinfo import summary
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingLR, SequentialLR, ConstantLR

# Import custom libraries
from utils.obj_detection_utils import collate_fn
from utils.segmentation_utils import display_image_with_mask, collapse_one_hot_mask, create_label_class_dict
from engines.segmentation import SegmentationEngine
from engines.schedulers import FixedLRSchedulerWrapper
from engines.loss_functions import DiceCrossEntropyLoss
from dataloaders.segmentation_dataloaders import ProcessDatasetSegmentation, SegmentationTransforms
from models.unet import create_unet

# Import custom libraries
from utils.common_utils import set_seeds
from utils.coco_dataset_utils import COCO_2_ImgMsk, split_dataset

# Warnings
import warnings
os.environ['TORCH_USE_CUDA_DSA'] = "1"
warnings.filterwarnings("ignore", category=UserWarning, module="torch.autograd.graph")
warnings.filterwarnings("ignore", category=FutureWarning, module="onnxscript.converter")

import torch._dynamo
#torch._dynamo.config.suppress_errors = True
torch._dynamo.config.cache_size_limit = 64
#warnings.filterwarnings("ignore", category=UserWarning, module="torch._dynamo")

# Create target model directory
MODEL_DIR = Path("outputs")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Set seeds
SEED = 42
set_seeds(SEED)

DOWNLOAD_COCO = False
PROCESS_COCO = False
VISUALIZE_TRANSFORMED_DATA = True

# 4. Specifying the Target Device

In [ ]:
# Activate cuda benchmark
#cudnn.benchmark = True

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

if device == "cuda":
    !nvidia-smi

# 5. Importing the Medical Imaging Dataset

This section prepares the training and validation datasets that will be used to train the U-Net model.

The complete dataset is publicly available in the [Kaggle UW-Madison GI Tract Image Segmentation Challenge](https://www.kaggle.com/competitions/uw-madison-gi-tract-image-segmentation/data). 

You must register on Kaggle and formally accept the competition terms in order to download the data.

After downloading the dataset, follow the instructions in Subsection 4.1. Once executed, the medical imaging dataset will be processed and organized into the following folder structure:

```
medical_imaging/
├── train/
│   └── images/
│       ├── img1.png
│       ├── img2.png
│       └── ...
│   └── masks/
│       ├── msk1.png
│       ├── msk2.png
│       └── ...
├── valid/
│   └── images/
│       ├── img1.png
│       ├── img2.png
│       └── ...
│   └── masks/
│       ├── msk1.png
│       ├── msk2.png
│       └── ...
```

## 5.1. Processing the Dataset

The script called `dataloaders/prepare_data_segmentation.py` takes as input parameters the path of the image database and the train.csv file that includes all the information needed to generate the target masks:
* The images are min-max normalized and saved in RGB PNG format in a specified target folder. If the image is grayscale, it is copied into each of the three channels.
* The target masks are created from the run-length code provided in ```bash train.csv``` (column ```python segmentation```).
* The script also allows for saving the images in a 2.5D format, where the first (R) channel is used to store the actual image 'i', the second (G) channel to store the image 'i + stride', and the third (B) channel to store the image 'i + 2*stride'.
* Additionally, the script offers the option to remove non-segmented slices. This allows the segmentation model to be trained using the entire dataset or only a subset containing segmented images.

The usage sample of this application is given next:

```bash
python .\prepare_data_segmentation.py -dimension 2d -csv data/train.csv -input_dir images/train -output_dir segmentation_data -remove_non_seg 1

```
More information about the input parameters is provided with the help parameter as follows:

```bash
python .\prepare_data_segmentation.py --help

usage: prepare_data_segmentation.py [-h] [-dimension {2d,2.5d}] [-stride STRIDE] [-csv CSV] [-input_dir INPUT_DIR]
                                    [-output_dir OUTPUT_DIR] [-test_patients TEST_PATIENTS]
                                    [-remove_non_seg REMOVE_NON_SEG] [-mask_rgb MASK_RGB]

options:
-h, --help                     Show this help message and exit
-dimension {2d,2.5d}           Choose either '2d' or '2.5d'
-stride STRIDE                 Specify the stride as an integer (default 1) for 2.5d
-csv CSV                       Path and file name of the csv file with rle data (default 'data/train.csv'
-input_dir INPUT_DIR           Specify the directory where the input images reside (default 'images/train')
-output_dir OUTPUT_DIR         Specify the directory where the images will be stored (default 'segmentation_data')
-test_patients VALID_PATIENTS  Specify the list of validation images for inference (default "['2', '6', '7', '9', '11', '15', '16', '140', '145', '146', '147', '148', '149', '154', '156']")
-remove_non_seg REMOVE_NON_SEG Remove pictures that are not segmented (default 1)
-mask_rgb MASK_RGB             Generate masks also in RGB format (default 0)
```

### Usage Example

In [ ]:
#!python .\prepare_data_segmentation.py -dimension 2d -csv data/train.csv -input_dir images/train -output_dir medical_imaging -remove_non_seg 1

## 5.2. Preparing Dataloaders

In [ ]:
# Create the dictionary describing label -> category), say in alphabetical order
target_categories={
    0: 'background',
    1: 'stomach',
    2: 'small_bowel',
    3: 'large_bowel'
}

# RGB
color_map = {
    0: (0, 0, 0),       # Background (Black)
    1: (232, 66, 66),   # Bus (Red)
    2: (35, 171, 75),   # Car (Green)
    3: (28, 163, 218),  # Truck (Blue)   
}

## Display categories
for cat_id, cat_name in target_categories.items():
    print(f"{cat_id}: {cat_name}")

In [ ]:
# The dataset contains two classes only: background and person
NUM_CLASSES = len(target_categories)
BATCH_SIZE = 4
ACCUM_STEPS = 8
IMG_SIZE = (384, 384)
AUGMENT_MAGNITUDE = 1 # 1 (low) to 5 (high)
THEME = 'light' # or 'dark'. Default is 'light'

# Define training, validation, and test data loaders
train_dataloader = torch.utils.data.DataLoader(
    dataset=ProcessDatasetSegmentation(
        root='data/medical_imaging/train',
        image_path="images",
        mask_path="masks",
        transforms=SegmentationTransforms(
            train=True,
            img_size=IMG_SIZE,
            mean_std_norm=True,
            augment_magnitude=AUGMENT_MAGNITUDE
            ),
        class_dictionary=target_categories
        ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory=True,
)

val_dataloader = torch.utils.data.DataLoader(
    dataset=ProcessDatasetSegmentation(
        root='data/medical_imaging/valid',
        image_path="images",
        mask_path="masks",
        transforms=SegmentationTransforms(
            train=False,
            img_size=IMG_SIZE,
            mean_std_norm=True,
            augment_magnitude=AUGMENT_MAGNITUDE
            ),
        class_dictionary=target_categories
        ),
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=True,
)

dataloaders = {
    'train':         train_dataloader,    
    'test':          val_dataloader
}

## 5.3. Visualizing Images with Masks

In [ ]:
if VISUALIZE_TRANSFORMED_DATA:

    # Visualize transformations
    BATCH_SIZE = 64
    # Train dataloader without transformations
    dataloader_nt = torch.utils.data.DataLoader(
        dataset=ProcessDatasetSegmentation(
            root='data/medical_imaging/train',
            image_path="images",
            mask_path="masks",
            transforms=SegmentationTransforms(
                train=False,
                img_size=IMG_SIZE,
                mean_std_norm=False,
                augment_magnitude=AUGMENT_MAGNITUDE
                ),
            class_dictionary=target_categories
            ), 
        batch_size=BATCH_SIZE,
        shuffle=False,
        pin_memory=True,
        collate_fn=collate_fn)
    
    # Test dataloader with transformations
    dataloader_t = torch.utils.data.DataLoader(
        dataset=ProcessDatasetSegmentation(
            root='data/medical_imaging/train',
            image_path="images",
            mask_path="masks",
            transforms=SegmentationTransforms(
                train=True,
                img_size=IMG_SIZE,
                mean_std_norm=False,
                augment_magnitude=AUGMENT_MAGNITUDE
                ),
            class_dictionary=target_categories
            ), 
        batch_size=BATCH_SIZE,
        shuffle=False,
        pin_memory=True,
        collate_fn=collate_fn)
    
    random.seed(SEED+1)

    # Visualize images and masks with and without transformations    
    for idx, ((img_nt, target_nt), (img_t, target_t)) in enumerate(zip(dataloader_nt, dataloader_t)):   

        # Pick random images
        random_indices = random.sample(range(BATCH_SIZE), min(10, BATCH_SIZE))
        for i in random_indices:

            # Set up the figure
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))
            
            # Pass subplot axes to the function
            mask_nt = collapse_one_hot_mask(target_nt[i])
            mask_t = collapse_one_hot_mask(target_t[i])

            # Create the label-class dictionary for the mask
            classes_nt = create_label_class_dict(target_nt[i], target_categories)
            classes_t = create_label_class_dict(target_t[i], target_categories)

            # Remove background, as it is always there
            classes_nt = dict(list(classes_nt.items())[1:])
            classes_t = dict(list(classes_t.items())[1:])

            # And generate the titles
            title_nt = f"Original: {', '.join(classes_nt.values())}"
            title_t = f"Transformed: {', '.join(classes_t.values())}"
            
            # Display overlaid images
            alpha, beta = 1.0, 0.5
            display_image_with_mask(img_nt[i], mask_nt, fig=fig, ax=axes[0], alpha=alpha, beta=beta, color_map=color_map, title=title_nt, theme=THEME)
            display_image_with_mask(img_t[i], mask_t, fig=fig, ax=axes[1], alpha=alpha, beta=beta, color_map=color_map, title=title_t, theme=THEME)
            
            plt.show() 
            
        if idx > -1:
            break

# 6. Creating the U-Net Architecture

In [ ]:
# Instantiate the model
model = create_unet(
    model_type="pretrained",
    backbone='convnext_large_384_in22ft1k', #'convnext_small_in22ft1k', #
    in_channels=3,
    num_classes=NUM_CLASSES,
    print_available_models=False,
    pretrained=True,
    encoder_freeze=False
)

# Unfreeze the base parameters
for parameter in model.parameters():
    parameter.requires_grad = True

#summary(model,
#        input_size=(BATCH_SIZE,3, IMG_SIZE[0], IMG_SIZE[1]),
#        col_names=["input_size", "output_size", "num_params", "trainable"],
#        col_width=20,
#        row_settings=["var_names"])

# 7. Training the Model

In [ ]:
# Train the model
EPOCHS = 35
LR = 1e-4
ETAMIN = 1e-6
model_type="model_seg_convnext"
model_name = model_type + ".pth"

In [ ]:
torch.cuda.empty_cache()

# Create AdamW optimizer
optimizer = torch.optim.AdamW(
    params=model.parameters(),
    lr=LR,
    betas=(0.9, 0.999),
    weight_decay=1e-4
    )

# Create loss function
#loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
loss_fn = DiceCrossEntropyLoss(
    num_classes=NUM_CLASSES,
    label_smoothing=0.1
    )

# Set scheduler
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#    optimizer, 
#    mode='min',
#    factor=0.1,
#    patience=3,
#)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS-5, eta_min=ETAMIN)

# And train...

# Instantiate the classification engine with the created model and the target device
engine = SegmentationEngine(
    model=model,                                # Model to be trained
    optimizer=optimizer,                        # Optimizer
    loss_fn=loss_fn,                            # Loss function
    scheduler=scheduler,                        # Scheduler     
    theme=THEME,                                # Theme
    log_verbose=True,                           # Verbosity
    device=device                               # Target device
    )

# Configure the training method
results = engine.train(
    target_dir=MODEL_DIR,                       # Directory where the model will be saved
    model_name=model_name,                      # Name of the model
    enable_resume=True,                         # Resume training from the last saved checkpoint
    save_best_model=[
        "last", "loss", "dice", "iou"],         # Save the best models based on different criteria
    keep_best_models_in_memory=False,           # If False: do not keep the models stored in memory for the sake of training time and memory efficiency
    dataloaders=dataloaders,                    # Dictionary with the dataloaders     
    apply_validation=True,                      # Enable validation step
    augmentation_strategy="always",             # Augmentation strategy        
    epochs=EPOCHS,                              # Total number of epochs
    amp=True,                                   # Enable Automatic Mixed Precision (AMP)
    enable_clipping=False,                      # Disable clipping on gradients, only useful if training becomes unestable
    debug_mode=False,                           # Disable debug mode    
    accumulation_steps=ACCUM_STEPS,             # Accumulation steps: effective batch size = batch_size x accumulation steps
    )

# 8. Making Predictions on Test

In [ ]:
from os.path import exists
def rename_model(model_name: str, new_name: str):
    old_name = model_name[0]
    os.rename(old_name, new_name)
    print(f"Renamed {old_name} to {new_name}")
    
# Find the model file with "model_1_loss_epoch" prefix and rename it
new_model_name = str(MODEL_DIR / f"{model_name}")
if not exists(new_model_name):
    model_name_dice = glob.glob(str(MODEL_DIR / f"{model_type}_dice_epoch*.pth"))
    rename_model(model_name_dice, new_model_name)

# Instantiate the classification engine with the created model and the target device
engine = SegmentationEngine(
    model=model,    
    log_verbose=True,
    device=device
    ).load(
        target_dir=MODEL_DIR,
        model_name=Path(new_model_name).name
    )

preds = engine.predict(
    dataloader=val_dataloader,
    num_classes=NUM_CLASSES,
    model_state="last", # This option takes the default loaded model
    output_type="onehot" #one-hot binary masks: (num_batches, batch_size, num_classes, W, H)
)

In [ ]:
print(f"Shape of the prediction tensor: {preds.shape}")

In [ ]:
# Visualize images and masks with and without transformations
BATCH_SIZE = val_dataloader.batch_size

# Create a test dataloader without mean/std transformation and augmentations
test_dataloader_nt = torch.utils.data.DataLoader(
    dataset=ProcessDatasetSegmentation(
        root='data/medical_imaging/valid',
        image_path="images",
        mask_path="masks",
        transforms=SegmentationTransforms(
            train=False, # If train if False, augmentation is not applied
            img_size=IMG_SIZE,
            mean_std_norm=False,
            augment_magnitude=AUGMENT_MAGNITUDE
            ),
        class_dictionary=target_categories
        ),
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=True,
)

for batch, (img, target) in enumerate(test_dataloader_nt):

    # Pick random images
    random_indices = random.sample(range(BATCH_SIZE), min(2, BATCH_SIZE))
    for i in random_indices:

        # Set up the figure
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Pass subplot axes to the function
        mask_true = collapse_one_hot_mask(target[i])
        mask_pred = collapse_one_hot_mask(preds[batch][i])

        # Create the label-class dictionary for the mask
        classes_true = create_label_class_dict(target[i], target_categories)
        classes_pred = create_label_class_dict(preds[batch][i], target_categories)

        # classes_true background, as it is always there
        classes_true = dict(list(classes_true.items())[1:])
        classes_pred = dict(list(classes_pred.items())[1:])

        # And generate the titles
        title_true = f"Ground-Truth: {', '.join(classes_true.values())}"
        title_pred = f"Predictions: {', '.join(classes_pred.values())}"
        
        # Display overlaid images
        alpha, beta = 1.0, 1.0
        display_image_with_mask(img[i], mask_true, fig=fig, ax=axes[0], alpha=alpha, beta=beta, color_map=color_map, title=title_true, theme=THEME)
        display_image_with_mask(img[i], mask_pred, fig=fig, ax=axes[1], alpha=alpha, beta=beta, color_map=color_map, title=title_pred, theme=THEME)
        
        plt.show() 
        
    if batch > 5:
        break